In [1]:
import numpy as np
import math
from matplotlib import pyplot as plt
from PIL import Image

In [2]:
im = Image.open("TestImage2c.jpg")
print(im.format, im.size, im.mode)

('JPEG', (1008, 756), 'RGB')


In [3]:
col, row = im.size
data = np.zeros((row * col, 3))
pixels = im.load()
print col, row
for i in range(row):
    for j in range(col):
        r, g, b = pixels[j, i]
        data[col * i + j, :] = r, g, b
grayscale = np.zeros(row * col)
for i in range(row):
    for j in range(col):
        grayscale[col * i + j] = data[col * i + j, 0] * 0.3 + data[col * i + j, 1] * 0.59 + data[col * i + j, 2] * 0.11
grayscale.resize((row, col))

1008 756


In [4]:
im = Image.fromarray(grayscale)
im.show()
# greyscale done

In [5]:
sobel = np.zeros((row, col, 2))
magnitude = np.zeros((row, col))
thita = np.zeros((row, col))
# Gx, Gy, G, thita
max = 0
min = 9999

threshold = 20
for i in range(1, row - 1):
    for j in range(1, col - 1):
        sobel[i, j, 0] = grayscale[i - 1, j + 1] + 2 * grayscale[i, j + 1] + grayscale[i + 1, j + 1] \
                         - grayscale[i - 1, j - 1] - 2 * grayscale[i, j - 1] - grayscale[i + 1, j - 1]
        sobel[i, j, 1] = grayscale[i - 1, j - 1] + 2 * grayscale[i - 1, j] + grayscale[i - 1, j + 1] \
                         - grayscale[i + 1, j - 1] - 2 * grayscale[i + 1, j] - grayscale[i + 1, j + 1]
        magnitude[i, j] = (sobel[i, j, 0] ** 2 + sobel[i, j, 1] ** 2) ** 0.5
        if sobel[i, j, 0] == 0:
            thita[i, j] = math.atan(sobel[i, j, 1] / 0.01)
        else:
            thita[i, j] = math.atan(sobel[i, j, 1] / sobel[i, j, 0])
        if magnitude[i, j] > max:
            max = magnitude[i, j]
        if magnitude[i, j] < min:
            min = magnitude[i, j]
print max, min
count = 0
for i in range(1, row - 1):
    for j in range(1, col - 1):
        magnitude[i, j] = magnitude[i, j] / max * 255
        if magnitude[i, j] > threshold:
            magnitude[i, j] = 255
            count += 1
        else:
            magnitude[i, j] = 0


952.571879598 0.0


In [6]:
im = Image.fromarray(magnitude)
im.show()
# greyscale done

In [7]:
expand_old = np.copy(magnitude)
expand_new = np.copy(magnitude)
for k in range(4):
    for i in range(1, row - 1):
        for j in range(1, col - 1):
            if expand_old[i-1,j]>0 or expand_old[i+1,j]>0 or \
            expand_old[i,j-1]>0 or expand_old[i,j+1]>0:
                expand_new[i,j] = 255
    expand_old = np.copy(expand_new)

In [8]:
im = Image.fromarray(expand_new)
im.show()

In [9]:
distance0 = np.copy(expand_new)
for i in range(1,row-1):
    for j in range(1,col-1):
        if distance0[i,j]>0:
            distance0[i,j]=1
distance_new = np.copy(distance0)
distance_old = np.copy(distance0)
for k in range(15):
    for i in range(1,row-1):
        for j in range(1,col-1):
            min=999
            if distance_old[i-1,j]<min:
                min = distance_old[i-1,j]
            if distance_old[i+1,j]<min:
                min = distance_old[i+1,j]
            if distance_old[i,j-1]<min:
                min = distance_old[i,j-1]
            if distance_old[i,j+1]<min:
                min = distance_old[i,j+1]
            distance_new[i,j]=distance0[i,j]+min
    distance_old = np.copy(distance_new)

for i in range(1,row-1):
    for j in range(1,col-1):
        if distance_old[i,j]<distance_old[i-1,j] or distance_old[i,j]<distance_old[i+1,j] or \
        distance_old[i,j]<distance_old[i,j-1] or distance_old[i,j]<distance_old[i,j+1]:
            distance0[i,j] = 0

count = 0
for i in range(1,row-1):
    for j in range(1,col-1):
        if distance0[i,j] > 0:
            distance0[i,j]=255
            count +=1

In [10]:
im = Image.fromarray(distance0)
im.show()
# greyscale done

In [11]:
point = np.zeros((count, 2))
count = 0
for i in range(1, row - 1):
    for j in range(1, col - 1):
        if distance0[i, j] == 255:
            point[count, :] = i, j
            count += 1
count

8299

In [12]:
# hough transform
angle = 720
p = 4000
hough = np.zeros((angle, p))
for i in range(count):
    if (i % 100)==0: print i
    for j in range(i + 1, count):
        x1, y1 = point[i, :]
        x2, y2 = point[j, :]
        if y1 == y2:
            tmpAngle = math.atan((x2 - x1) / 0.001)
        else:
            tmpAngle = math.atan((x2 - x1) / (y1 - y2))
        tmpP = x1 * math.cos(tmpAngle) + y1 * math.sin(tmpAngle)
        tmpAngle = tmpAngle / (2 * math.pi) * 360
        if tmpAngle < 0: tmpAngle = tmpAngle + 360
        tmpAngle = int(tmpAngle * 2)
        tmpP = int(tmpP)+2000
        hough[tmpAngle, tmpP] += 1
# get original hough array

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200


In [13]:
for i in range(angle):
    for j in range(p):
        if hough[i, j] > threshold:
            count += 1
print count
effectiveHough = np.zeros((count, 3))
count = 0
for i in range(0, angle):
    for j in range(0, p):
        tmp = hough[i, j]
        top = 0 if i - 3 < 0 else i - 3
        bottom = angle - 1 if i + 3 > angle - 1 else i + 3
        left = 0 if j - 10 < 0 else j - 10
        right = p - 1 if j + 10 > p - 1 else j + 10
        maxFlag = 1
        for k in range(top, bottom + 1):
            for l in range(left, right + 1):
                if tmp < hough[k, l]:
                    maxFlag = 0
        if (tmp > threshold) and maxFlag == 1:
            effectiveHough[count, :] = hough[i, j], i, j
            count += 1
print count
#get effectiveHough

# for i in range(100):
#     print effectiveHough[i,0]

for i in range(count):
    for j in range(i + 1, count):
        if effectiveHough[i, 0] < effectiveHough[j, 0]:
            tmpNum, tmpAngle, tmpP = effectiveHough[i, :]
            effectiveHough[i] = effectiveHough[j]
            effectiveHough[j] = tmpNum, tmpAngle, tmpP

#sort effectiveHough

182824
900


In [14]:
for i in range(100):
    print effectiveHough[i]

[ 11271.    110.   2711.]
[ 10620.     71.   2838.]
[ 10375.    682.   2424.]
[ 9428.   683.  2328.]
[ 7795.    73.  2902.]
[ 7079.    83.  2330.]
[ 5944.   106.  2659.]
[ 5837.   688.  2099.]
[ 5834.   702.  2238.]
[ 4822.   705.  2183.]
[ 4451.   717.  2358.]
[  4.26000000e+03   2.00000000e+00   2.31800000e+03]
[ 4186.   707.  2275.]
[ 4150.   710.  2301.]
[ 3455.   700.  2142.]
[ 3240.     0.  2363.]
[ 3191.     0.  2379.]
[ 3191.    13.  2405.]
[ 3123.   715.  2343.]
[ 2824.   174.  2472.]
[ 2686.   718.  2313.]
[ 2626.     8.  2374.]
[ 2518.    50.  2431.]
[ 2478.   709.  2373.]
[ 2465.   179.  2447.]
[ 2419.    67.  2415.]
[ 2391.   719.  2378.]
[ 2345.     4.  2338.]
[ 2215.    45.  2409.]
[ 2191.    18.  2410.]
[ 2186.     0.  2291.]
[ 2116.   683.  2000.]
[ 2017.    11.  2362.]
[ 1971.    69.  2453.]
[ 1967.    39.  2381.]
[ 1963.    59.  2405.]
[ 1910.    78.  2447.]
[ 1906.   101.  2516.]
[ 1889.    21.  2425.]
[ 1865.    90.  2450.]
[ 1834.   150.  2577.]
[ 1824.   718.  22

In [15]:
resultImage = np.copy(magnitude)
for i in range(row):
    for j in range(col):
        for k in range(10):
            if int(i * math.cos(effectiveHough[k, 1] / 720.0 * 2 * math.pi) +
                                   j * math.sin(effectiveHough[k, 1] / 720.0 * 2 * math.pi)) - effectiveHough[
                k, 2] +2000 == 0:
                resultImage[i, j] = 255

In [16]:
im = Image.fromarray(resultImage)
im.show()

### effectiveHough 数组中的内容：
* 落在该直线上点的数量
* thita:角度，取值0-720
* p:直线到原点距离，取值-2000-2000

#### 该数组已sort

### 直线——线段
* 对图中所有点搜索
* effectiveLinePoint[100,max(effectiveHough[0]),2],如果点在前一百的hough直线上，加入到数组中
* 对数组作sort
* 求出数组中的线段

In [35]:
maxEffectvieHough = int(effectiveHough[0,0])
effectiveLinePoint = np.zeros((100,maxEffectvieHough,2))
lineCount = np.zeros(100)
for i in range(row):
    for j in range(col):
        for k in range(100):
            onLine = int(i * math.cos(effectiveHough[k, 1] / 720.0 * 2 * math.pi) + 
                    j * math.sin(effectiveHough[k, 1] / 720.0 * 2 * math.pi)) - effectiveHough[k, 2] +2000 
            if (onLine==0) and (magnitude[i,j]==255):
                tmp = int(lineCount[k])
                effectiveLinePoint[k,tmp,:]=i,j
                lineCount[k]+=1
for i in range (100):
    print lineCount[i]

266.0
329.0
374.0
379.0
258.0
237.0
135.0
50.0
140.0
67.0
184.0
137.0
251.0
194.0
100.0
188.0
190.0
142.0
132.0
135.0
153.0
135.0
159.0
124.0
62.0
99.0
172.0
152.0
158.0
126.0
121.0
140.0
132.0
173.0
141.0
122.0
127.0
123.0
109.0
83.0
127.0
107.0
102.0
112.0
118.0
80.0
74.0
114.0
133.0
146.0
144.0
89.0
101.0
114.0
106.0
143.0
115.0
138.0
96.0
173.0
98.0
129.0
155.0
105.0
136.0
123.0
101.0
128.0
107.0
82.0
68.0
133.0
104.0
65.0
90.0
86.0
83.0
187.0
111.0
179.0
86.0
49.0
120.0
87.0
94.0
110.0
153.0
117.0
122.0
107.0
153.0
91.0
75.0
117.0
107.0
94.0
84.0
129.0
86.0
108.0
